<a href="https://colab.research.google.com/github/MarceloJSSantos/bootcamp-bradesco-genai-dados-desadio-assistente-voz-whisper-gemini/blob/main/Desafio_Bootcamp_Bradesco_IAGen_Dados_AssistenteVozWhisperEGemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Desafio do Bootcamp Badesco/DIO GenAI e Dados

In [61]:
IDIOMA = 'pt'

In [62]:
# @title
# Código JavaScript para gravar áudio do usuário"
SCRIPT_JS_RECORD = """
async function record(silence_duration = 1500) {
  // Desativa filtros automáticos que amplificam o ruído de fundo
  const stream = await navigator.mediaDevices.getUserMedia({
    audio: {
      echoCancellation: false,
      echoCancellation: false,
      noiseSuppression: false,
      autoGainControl: false
    }
  });

  const mediaRecorder = new MediaRecorder(stream);
  const audioChunks = [];

  const audioContext = new AudioContext();
  const source = audioContext.createMediaStreamSource(stream);
  const analyser = audioContext.createAnalyser();
  analyser.fftSize = 512;
  source.connect(analyser);

  const dataArray = new Uint8Array(analyser.frequencyBinCount);
  let silenceStart = performance.now();
  let isRecording = true;

  // --- CONFIGURAÇÃO DE SENSIBILIDADE ---
  // Se o silêncio não for detectado, aumente este valor (ex: para 25 ou 30)
  const THRESHOLD = 25;

  mediaRecorder.ondataavailable = (event) => audioChunks.push(event.data);

  const stopRecording = () => {
    return new Promise((resolve) => {
      mediaRecorder.onstop = () => {
        const audioBlob = new Blob(audioChunks);
        const reader = new FileReader();
        reader.readAsDataURL(audioBlob);
        reader.onloadend = () => resolve(reader.result);
      };
      mediaRecorder.stop();
      stream.getTracks().forEach(track => track.stop());
    });
  };

  mediaRecorder.start();

  return new Promise((resolve) => {
    const checkAudio = () => {
      if (!isRecording) return;

      analyser.getByteFrequencyData(dataArray);

      // Calcula o volume médio das frequências de voz
      let sum = 0;
      for (let i = 0; i < dataArray.length; i++) {
        sum += dataArray[i];
      }
      let average = sum / dataArray.length;

      // Log para você ver no console do navegador (F12) o nível do seu silêncio
      // console.log("Volume atual:", average);

      if (average < THRESHOLD) {
        if (performance.now() - silenceStart > silence_duration) {
          isRecording = false;
          resolve(stopRecording());
          return;
        }
      } else {
        silenceStart = performance.now(); // Reset se houver som
      }

      requestAnimationFrame(checkAudio);
    };

    checkAudio();
  });
}
"""

1) Gravar Áudio Com Python e JavaScript

In [63]:
# Referência: https://gist.github.com/korakot/c21c3476c024ad6d56d5f48b0bca92be

from IPython.display import Audio, display, Javascript
from google.colab import output
from base64 import b64decode

import os

def gravador(tempo_silencio_sec=2):
  # Executa o código JavaScript que captura o audio a ser gravado
  display(Javascript(SCRIPT_JS_RECORD))
  # Recebe o áudio gravado como resultado do JavaScript
  js_result = output.eval_js('record(%s)' % (tempo_silencio_sec * 1000))
   # Decodifica o áudio em base64
  decode_audio = b64decode(js_result.split(',')[1])
  # Define o nome da pasta
  pasta = "audios"

  # Verifica se a pasta já existe antes de criar
  if not os.path.exists(pasta):
      os.makedirs(pasta)

  # Salva o áudio em um arquivo
  file_name = f'{pasta}/audio_capturado.mp3'

  with open(file_name, 'wb') as f:
    f.write(decode_audio)
  # Retorna o caminho do arquivo de áudio (pasta padrão do Google Colab)
  return f'/content/{file_name}'

# Grava o áudio do usuário por um tempo determinado (padrão 5 segundos)
print('Faça uma pergunta ao Gemini...\n')
arquivo_audio_entrada = gravador()
print("Gravação finalizada automaticamente pelo silêncio!\n")

# Exibe o áudio gravado
display(Audio(arquivo_audio_entrada, autoplay=False))


Faça uma pergunta ao Gemini...



<IPython.core.display.Javascript object>

Gravação finalizada automaticamente pelo silêncio!



2) Reconhecimento de Fala com Whisper (OpenAI)

In [64]:
# Tenta importar a biblioteca, mas cado tenha um erro, pois não está instalado
# Instala a biblioteca e então, importa
try:
    import whisper
except ImportError:
    print("Instalando Whisper...")
    !pip install git+https://github.com/openai/whisper.git -q
    import whisper

In [65]:
# Selecione o modelo do Whisper que melhor atenda às suas necessidades:
# https://github.com/openai/whisper#available-models-and-languages
model = whisper.load_model("small")

# Transcreve o audio gravado anteriormente.
result = model.transcribe(arquivo_audio_entrada, fp16=False, language=IDIOMA)
transcricao_whisper = result["text"]
print(transcricao_whisper)

 Relacione para mim de maneira sintetizada as estações do ano.


3) Integração com a API do Gemini (por ser gratuito)

In [66]:
# Tenta importar a biblioteca, mas cado tenha um erro, pois não está instalado
# Instala a biblioteca e então, importa
try:
    import google.generativeai as genai
except ImportError:
    print("Instalando genai...")
    !pip install -q -U google-generativeai
    import google.generativeai as genai

In [67]:
# Substitua pela sua chave real do AI Studio
minha_chave = "COLOQUE SUA CHAVE GEMINI"
genai.configure(api_key=minha_chave)

# 1. Usando o modelo mais atual (2.0 Flash é o padrão agora)
# Se der erro, você pode tentar 'gemini-1.5-flash' sem o v1beta no nome
model = genai.GenerativeModel('gemini-2.5-flash')

try:
    # 2. Gerando o conteúdo
    resposta_gemini = model.generate_content(transcricao_whisper)
    resposta_gemini_texto = resposta_gemini.text.replace("*", "")

    # 3. Exibindo o texto
    print("Resposta do Gemini:")
    print(resposta_gemini_texto)

except Exception as e:
    print(f"Ocorreu um erro: {e}")
    # Caso o modelo 2.0 ainda não esteja na sua região, listamos os disponíveis:
    print("\nModelos que você pode usar:")
    for m in genai.list_models():
        if 'generateContent' in m.supported_generation_methods:
            print(m.name)

Resposta do Gemini:
As estações do ano são quatro períodos distintos que se repetem anualmente, causados principalmente pela inclinação do eixo da Terra em relação ao Sol, o que gera variações na intensidade da luz solar e na duração dos dias.

Aqui está uma síntese de cada uma:

1.  Primavera:
       Característica: Transição do frio para o calor.
       Marcas: Renascimento da natureza, florescimento de plantas, dias que se alongam e temperaturas amenas.

2.  Verão:
       Característica: Período mais quente do ano.
       Marcas: Dias longos, muita luz solar, temperaturas elevadas e propício para atividades ao ar livre.

3.  Outono:
       Característica: Transição do calor para o frio.
       Marcas: As folhas das árvores mudam de cor e caem (em climas temperados), dias começam a encurtar, temperaturas a baixar e época de colheita.

4.  Inverno:
       Característica: Estação mais fria do ano.
       Marcas: Dias curtos, noites longas, temperaturas baixas, podendo haver neve ou gea

4) Transformando a Resposta do Gemini em Voz com edge_tts (versão da voz gratuita + natural)

In [68]:
# Tenta importar a biblioteca, mas cado tenha um erro, pois não está instalado
# Instala a biblioteca e então, importa
try:
    import edge_tts
    import asyncio
except ImportError:
    print("Instalando edge-tts...")
    !pip install edge-tts
    import edge_tts
    import asyncio

In [69]:
response_audio = '/content/audios/audio_sintetizado.mp3'

async def sintetizar():
    VOZ = "pt-BR-FranciscaNeural" # Ou "pt-BR-AntonioNeural"

    communicate = edge_tts.Communicate(resposta_gemini_texto, VOZ)

    await communicate.save(response_audio)

# No Colab, usamos o loop de eventos atual
await sintetizar()

# Para ouvir no Colab:
from IPython.display import Audio
Audio(response_audio, autoplay=True)